In [1]:
!pip install fuzzywuzzy

In [2]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.3 MB/s eta

In [9]:
!pip install gradio
!pip install transformers

In [4]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
import pandas as pd

import gradio as gr
import tempfile
from transformers import pipeline


In [14]:
import pandas as pd
import gradio as gr
import tempfile
from fuzzywuzzy import fuzz

def handle_file(file):
  """Reads the uploaded XLSX file, performs fuzzy matching, and returns the modified file path."""
  if not file:
    return None  # Handle cases where no file is uploaded

  try:
    df = pd.read_excel(file)
    columns = df.columns[:2]
    df['New_Ratio'] = df.apply(lambda row: fuzz.ratio(row[columns[0]], row[columns[1]]), axis=1)

    # Create a temporary file for download
    with tempfile.NamedTemporaryFile(prefix='Customfile',suffix='.xlsx', delete=False) as temp_file:
      df.to_excel(temp_file.name, index=False)
      temp_file_path = temp_file.name

    return temp_file_path , df

  except FileNotFoundError:
    return "Error: File not found. Please upload a valid XLSX file."
  except Exception as e:
    return f"Error: {e}"  # Catch other potential exceptions

file_input = gr.components.File(label="Upload XLSX File")
download_button = gr.File(label="Download Modified File")
dataframe_output = gr.components.Dataframe(type="pandas")
def download_modified_file():
  """Downloads the modified file and returns the filename for the button label."""
  temp_file_path = interface.get_output("handle_file")  # Access temporary file path
  filename = os.path.basename(temp_file_path)  # Extract filename from path using os.path.basename
  return gr.File.from_path(temp_file_path, label=filename)  # Set filename as button label

interface = gr.Interface(
  fn=handle_file,
  inputs=file_input,
  outputs=[download_button , dataframe_output ],
  title="XLSX Fuzzy Matching with Download",
  description="Upload an XLSX file to calculate fuzzy similarity ratios and download the modified file."
)

interface.launch(inline=False)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4d9b910b288c3ec033.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
